In [23]:
import cv2
import numpy as np 
import math

In [24]:
def generate_data():
    # 本函数生成0-9，10个数字的图片矩阵
    image_data=[]
    num_0 = np.array(
    [[0,0,1,1,0,0],
    [0,1,0,0,1,0],
    [0,1,0,0,1,0],
    [0,1,0,0,1,0],
    [0,0,1,1,0,0],
    [0,0,0,0,0,0]])
    image_data.append(num_0)
    num_1 = np.array(
    [[0,0,0,1,0,0],
    [0,0,1,1,0,0],
    [0,0,0,1,0,0],
    [0,0,0,1,0,0],
    [0,0,1,1,1,0],
    [0,0,0,0,0,0]])
    image_data.append(num_1)
    num_2 = np.array(
    [[0,0,1,1,0,0],
    [0,1,0,0,1,0],
    [0,0,0,1,0,0],
    [0,0,1,0,0,0],
    [0,1,1,1,1,0],
    [0,0,0,0,0,0]])
    image_data.append(num_2)
    num_3 = np.array(
    [[0,0,1,1,0,0],
    [0,0,0,0,1,0],
    [0,0,1,1,0,0],
    [0,0,0,0,1,0],
    [0,0,1,1,0,0],
    [0,0,0,0,0,0]])
    image_data.append(num_3)
    num_4 = np.array(
    [
    [0,0,0,0,1,0],
    [0,0,0,1,1,0],
    [0,0,1,0,1,0],
    [0,1,1,1,1,1],
    [0,0,0,0,1,0],
    [0,0,0,0,0,0]])
    image_data.append(num_4)
    num_5 = np.array(
    [
    [0,1,1,1,0,0],
    [0,1,0,0,0,0],
    [0,1,1,1,0,0],
    [0,0,0,0,1,0],
    [0,1,1,1,0,0],
    [0,0,0,0,0,0]])
    image_data.append(num_5)
    num_6 = np.array(
    [[0,0,1,1,0,0],
    [0,1,0,0,0,0],
    [0,1,1,1,0,0],
    [0,1,0,0,1,0],
    [0,0,1,1,0,0],
    [0,0,0,0,0,0]])
    image_data.append(num_6)
    num_7 = np.array(
    [
    [0,1,1,1,1,0],
    [0,0,0,0,1,0],
    [0,0,0,1,0,0],
    [0,0,0,1,0,0],
    [0,0,0,1,0,0],
    [0,0,0,0,0,0]])
    image_data.append(num_7)
    num_8 = np.array(
    [[0,0,1,1,0,0],
    [0,1,0,0,1,0],
    [0,0,1,1,0,0],
    [0,1,0,0,1,0],
    [0,0,1,1,0,0],
    [0,0,0,0,0,0]])
    image_data.append(num_8)
    num_9 = np.array(
    [[0,0,1,1,1,0],
    [0,1,0,0,1,0],
    [0,0,1,1,1,0],
    [0,1,0,0,1,0],
    [0,0,0,0,1,0],
    [0,0,0,0,0,0]])
    image_data.append(num_9)
    
    return image_data




In [25]:
def get_feature(img):  
    img = img/255 # 归一化
    g_img = np.zeros((img.shape[0],img.shape[1],2))
    for i in range(1,img.shape[0]-1):
        for j in range(1,img.shape[1]-1):
            gx = img[i+1,j] - img[i-1,j]
            gy = img[i,j+1] - img[i,j-1]
            g = (gx**2 + gy**2)**0.5
            if gx == 0 and gy == 0:
                dg = 0
            elif gx == 0 and gy != 0:
                dg = math.pi/2
            else:
                dg = math.atan(gy/gx) 
                if dg < 0:
                    dg = dg + math.pi
            if dg == math.pi:
                dg = 0
            g_img[i,j,0] = g
            g_img[i,j,1] = dg 
    cell_n = np.zeros((4))
    #cell h
    h = img.shape[0]//3
    #cell w
    w = img.shape[1]//3
    #cell size per h
    h_size = 3
    #cell size per w
    w_size = 3
    cell = np.zeros((h,w,4))    
    for m in range(h):
        for n in range(w):
            for i in range(h_size*m,h_size*(m+1)):
                for j in range(w_size*n,w_size*(n+1)):
                    cell_n[int(g_img[i,j,1]//(math.pi/4))] += g_img[i,j,0]
            cell[m,n] = cell_n
    block = np.zeros((h//2,w//2,4))
    for p in range(h//2):
        for q in range(w//2):
            for i in range(2*p,2*p+2):
                for j in range(2*q,2*q+2):
                    block[p,q] += cell[i,j]
    block_norm = np.zeros((h//2,w//2,4))
    for i in range(h//2):
        for j in range(w//2):
            length = (np.linalg.norm(block[i,j])**2 + 0.000001)**0.5
            block_norm[i,j] = block[i,j]/length
    block_norm = block_norm.reshape(block_norm.shape[0]*block_norm.shape[1],4)
    return block_norm 

In [26]:
def model(features, x):
    test_feature = get_feature(x)
    #print(test_feature)
    test_feature = test_feature.ravel()
    dists = []
    for f in features:
        f = f.ravel()
        dist = np.sqrt(sum(np.power((f - test_feature), 2)))
        dists.append(dist)
    #print(dists)
    #print(min(dists))
    res = dists.index(min(dists))
    return res

In [27]:
# 得到10个数字的特征数据
image_data = generate_data()
features = []
for img in image_data:
    feature = get_feature(img)
    features.append(feature)
    print(feature)

[[0.51982957 0.44109002 0.51982957 0.51460502]]
[[0.04291154 0.12137216 0.94405384 0.3034304 ]]
[[0.41599625 0.84977755 0.32355264 0.        ]]
[[0.         0.06136788 0.99805388 0.        ]]
[[0.61971583 0.63738947 0.45070242 0.07967368]]
[[0.         0.09511631 0.73983124 0.66581415]]
[[0.30898908 0.07282942 0.87546905 0.36414712]]
[[0.73893865 0.20900341 0.6404135  0.        ]]
[[0.         0.0520204  0.99316736 0.1040408 ]]
[[0.15388359 0.14508284 0.97459605 0.07254142]]


In [28]:
# 原始的数字5，像素点值整体右移一位进行验证
test_5 = np.array(
    [
    [0,0,1,1,1,0],
    [0,0,1,0,0,0],
    [0,0,1,1,1,0],
    [0,0,0,0,0,1],
    [0,0,1,1,1,0],
    [0,0,0,0,0,0]])

# 原始的数字0，整体向右移一位
test_0 = np.array(
    [[0,0,0,1,1,0],
    [0,0,1,0,0,1],
    [0,0,1,0,0,1],
    [0,0,1,0,0,1],
    [0,0,0,1,1,0],
    [0,0,0,0,0,0]])

### 该方法提取特征以后泛化性还是比较低

In [21]:
res_5 = model(features, image_data[5])
print("原始图片5的预测结果为:{}".format(res_5))

res_5 = model(features, test_5)
print("测试图片5的预测结果为:{}".format(res_5))

res_0 = model(features, image_data[0])
print("原始图片0的预测结果为:{}".format(res_0))
res_0 = model(features, test_0)
print("测试图片0的预测结果为:{}".format(res_0))   


原始图片5的预测结果为:5
测试图片5的预测结果为:1
原始图片0的预测结果为:0
测试图片0的预测结果为:6
